In [1]:
%%capture
!pip install transformers

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import transformers
from transformers import AutoTokenizer
from transformers import  DistilBertForTokenClassification

from torch.optim import AdamW

import torch
import torch.nn as nn
from torch.optim import SGD
import torch.nn.functional as F
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score


In [2]:
import pandas as pd
df=pd.read_excel('../Data/rawsmsdata.xlsx')
df

,sentence
0,AccountNo XXXXXX3406 CREDIT with amount Rs.110...
1,Rs.17.70 Debited to SB-xxx9488 AcBal 489.60 CL...
2,"UPDATE INR:32,241.00 debited from HDFC Bank XX..."
3,Debited INR.500.00 from A/c X2731 on 08-MAR-20...
4,INR.500.00 withdrawn via card 6530 on 08/03/23...
5,ICICI Bank Acc XX424 debited with INR 500.00 o...
6,Rs.500.00 withdrawn at ATM ID:IOBD5123 SRIRAM ...
7,INR 500.00 debited from A/c XXXXXXXX3885 towar...
8,Dear Customer statement for YES BANK Credit Ca...
9,INR 179.00 spent on ICICI Bank Card XX1008 on ...


In [4]:
import json
f=open('../Data/annotations.json')
annot = json.load(f)

In [5]:
import pandas as pd

data = pd.DataFrame(annot['annotations'][1:], columns=['text', 'labels'])
data


,text,labels
0,AccountNo XXXXXX3406 CREDIT with amount Rs.110...,"{'entities': [[0, 9, 'OTHER'], [10, 20, 'ACCOU..."
1,Rs.17.70 Debited to SB-xxx9488 AcBal 489.60 CL...,"{'entities': [[0, 8, 'DEBIT_AMOUNT'], [9, 19, ..."
2,"""UPDATE INR:32,241.00 debited from HDFC Bank X...","{'entities': [[0, 7, 'OTHER'], [8, 21, 'DEBIT_..."
3,"""Debited INR.500.00 from A/c X2731 on 08-MAR-2...","{'entities': [[0, 8, 'OTHER'], [9, 19, 'DEBIT_..."
4,INR.500.00 withdrawn via card 6530 on 08/03/23...,"{'entities': [[0, 10, 'DEBIT_AMOUNT'], [11, 29..."
5,ICICI Bank Acc XX424 debited with INR 500.00 o...,"{'entities': [[0, 10, 'BANK_NAME'], [11, 14, '..."
6,Rs.500.00 withdrawn at ATM ID:IOBD5123 SRIRAM ...,"{'entities': [[0, 9, 'DEBIT_AMOUNT'], [10, 22,..."
7,INR 500.00 debited from A/c XXXXXXXX3885 towar...,"{'entities': [[0, 10, 'DEBIT_AMOUNT'], [11, 27..."
8,Dear Customer statement for YES BANK Credit Ca...,"{'entities': [[0, 27, 'OTHER'], [28, 36, 'BANK..."
9,"""INR 179.00 spent on ICICI Bank Card XX1008 on...","{'entities': [[0, 1, 'OTHER'], [1, 11, 'DEBIT_..."


In [7]:
## Creating labels for the text from the custom ner tags created with annotations.json file.

import pandas as pd
import json

def extract_labels(file_path):
    f=open(file_path)
    annot = json.load(f)
    df1 = pd.DataFrame(columns = ["text","labels"])
    
    data = pd.DataFrame(annot['annotations'][1:], columns=['text', 'labels'])
    
    st_list =[]
    text_list=[]
    for i in range(len(data)):
        lab = data['labels'].values[i]
        text = data['text'].values[i]
        out = []
        for i in lab['entities']:
            if i[2] == "OTHER":
                ln = len(text[i[0]: i[1]].split())
                for j in range(ln):
                    out.append('O')
            else:
                out.append(i[2])
        st =' '.join(out)
        text_list.append(text)

        st_list.append(st)
        
    df1['text'] = text_list
    df1['labels'] = st_list
    df1 = df1.iloc[:-1]
    return df1
file_path = '../Data/annotations.json'
extract_labels(file_path)
df2 = extract_labels(file_path)
df2

,text,labels
0,AccountNo XXXXXX3406 CREDIT with amount Rs.110...,O ACCOUNT_NO O O O CREDIT_AMOUNT O TRANSACTION...
1,Rs.17.70 Debited to SB-xxx9488 AcBal 489.60 CL...,DEBIT_AMOUNT O O ACCOUNT_NO O BALANCE_AMOUNT O...
2,"""UPDATE INR:32,241.00 debited from HDFC Bank X...",O DEBIT_AMOUNT O O BANK_NAME ACCOUNT_NO O TRAN...
3,"""Debited INR.500.00 from A/c X2731 on 08-MAR-2...",O DEBIT_AMOUNT O O ACCOUNT_NO O TRANSACTION_DT...
4,INR.500.00 withdrawn via card 6530 on 08/03/23...,DEBIT_AMOUNT O O O CARD_NO O TRANSACTION_DT O ...
5,ICICI Bank Acc XX424 debited with INR 500.00 o...,BANK_NAME O ACCOUNT_NO O O DEBIT_AMOUNT O TRAN...
6,Rs.500.00 withdrawn at ATM ID:IOBD5123 SRIRAM ...,DEBIT_AMOUNT O O ATM_ID O O O O TRANSACTION_DT...
7,INR 500.00 debited from A/c XXXXXXXX3885 towar...,DEBIT_AMOUNT O O O ACCOUNT_NO O O O O O O O O ...
8,Dear Customer statement for YES BANK Credit Ca...,O O O O BANK_NAME O O O CREDIT_CARD_NO O O O O...
9,"""INR 179.00 spent on ICICI Bank Card XX1008 on...",O DEBIT_AMOUNT O O BANK_NAME O DEBIT_CARD_NO O...


In [ ]:
import pandas as pd
import json

def extract_labels(file_path):
    with open(file_path) as f:
        annot = json.load(f)
    
    data = annot['annotations'][1:]
    
    df = pd.DataFrame(data, columns=['text', 'labels'])
    
    df['labels'] = df['labels'].apply(lambda x: ' '.join(['O' if entity[2] == 'OTHER' else entity[2] for entity in x['entities']]))
    
    return df


In [8]:
import numpy as np
df2.rename(columns = {'text':'sentence', 'labels':'tags'}, inplace = True)

#split train, dev , test sets
df_train, df_dev, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

In [ ]:

from transformers import AutoTokenizer, DistilBertModel
from transformers import DistilBertTokenizer

class DistilbertNER(nn.Module):
  """
  Implement NN class based on distilbert pretrained from Hugging face.
  Inputs : 
    tokens_dim : int specifyng the dimension of the classifier
  """
  
  def __init__(self, tokens_dim):
    super(DistilbertNER,self).__init__()
    
    if type(tokens_dim) != int:
            raise TypeError('Please tokens_dim should be an integer')

    if tokens_dim <= 0:
          raise ValueError('Classification layer dimension should be at least 1')

    self.pretrained = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels = tokens_dim) #set the output of each token classifier = unique_lables


  def forward(self, input_ids, attention_mask, labels = None): #labels are needed in order to compute the loss
    """
  Forwad computation of the network
  Input:
    - inputs_ids : from model tokenizer
    - attention :  mask from model tokenizer
    - labels : if given the model is able to return the loss value
  """

    #inference time no labels
    if labels == None:
      out = self.pretrained(input_ids = input_ids, attention_mask = attention_mask )
      return out

    out = self.pretrained(input_ids = input_ids, attention_mask = attention_mask , labels = labels)
    return out